# IMPORT LIBRARIES

In [29]:
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# LOAD df

In [11]:
# Load the df
file_path = 'dataset.csv'
df = pd.read_csv(file_path)


In [12]:
# Display the first few rows of the df
df.head()

,ga:sourceMedium,ga:pageTitle,ga:users,ga:bounceRate,ga:pageviews,ga:pageviewsPerSession,ga:avgTimeOnPage
0,facebook / cpc,"Bikin Lambe Turah Diomelin, Nih Si Seksi yang ...",13736.0,19.269967,83593.0,6.042140,21.790189
1,facebook / cpc,Heboh! Kiki Farel Transfer Uang Jajan Rp 1 Jut...,15909.0,62.303763,74032.0,4.612011,16.330553
2,facebook / cpc,"Tak Sesuai Harapan, Barang Belanja Online Ini ...",7748.0,30.934520,54657.0,6.949396,21.429338
3,facebook / cpc,"Antar 'Tuyul', 7 Pengemudi Grab Dibekuk",43893.0,6.966219,47924.0,1.106560,5.056397
4,facebook / cpc,Melihat Kantor Pelayanan Grab Driver Center,41155.0,11.630219,44072.0,43.809145,180.944233


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ga:sourceMedium         156 non-null    object 
 1   ga:pageTitle            156 non-null    object 
 2   ga:users                155 non-null    float64
 3   ga:bounceRate           155 non-null    float64
 4   ga:pageviews            155 non-null    float64
 5   ga:pageviewsPerSession  155 non-null    float64
 6   ga:avgTimeOnPage        155 non-null    float64
dtypes: float64(5), object(2)
memory usage: 8.7+ KB


## Replace Column Name

In [14]:
df.columns = df.columns.str.replace('ga:', '')
df

,sourceMedium,pageTitle,users,bounceRate,pageviews,pageviewsPerSession,avgTimeOnPage
0,facebook / cpc,"Bikin Lambe Turah Diomelin, Nih Si Seksi yang ...",13736.0,19.269967,83593.0,6.042140,21.790189
1,facebook / cpc,Heboh! Kiki Farel Transfer Uang Jajan Rp 1 Jut...,15909.0,62.303763,74032.0,4.612011,16.330553
2,facebook / cpc,"Tak Sesuai Harapan, Barang Belanja Online Ini ...",7748.0,30.934520,54657.0,6.949396,21.429338
3,facebook / cpc,"Antar 'Tuyul', 7 Pengemudi Grab Dibekuk",43893.0,6.966219,47924.0,1.106560,5.056397
4,facebook / cpc,Melihat Kantor Pelayanan Grab Driver Center,41155.0,11.630219,44072.0,43.809145,180.944233
...,...,...,...,...,...,...,...
151,facebook / cpc,"Edan, Ponsel Ini Punya Baterai 13.000 mAh",198.0,76.923077,215.0,1.837607,101.685393
152,facebook / cpc,Gempa Keras di Jakarta Terasa Lebih dari 1 Menit,180.0,20.000000,215.0,2.388889,233.640000
153,facebook / cpc,Jennifer Dunn dan Deretan Mobil Mewahnya,180.0,100.000000,215.0,23.888889,97.836735
154,facebook / cpc,Kasus Salah Baca Xiaomi Terulang Kembali,180.0,83.333333,215.0,1.990741,178.449438


In [16]:
df

,sourceMedium,pageTitle,users,bounceRate,pageviews,pageviewsPerSession,avgTimeOnPage
0,facebook / cpc,"Bikin Lambe Turah Diomelin, Nih Si Seksi yang ...",13736.0,19.269967,83593.0,6.042140,21.790189
1,facebook / cpc,Heboh! Kiki Farel Transfer Uang Jajan Rp 1 Jut...,15909.0,62.303763,74032.0,4.612011,16.330553
2,facebook / cpc,"Tak Sesuai Harapan, Barang Belanja Online Ini ...",7748.0,30.934520,54657.0,6.949396,21.429338
3,facebook / cpc,"Antar 'Tuyul', 7 Pengemudi Grab Dibekuk",43893.0,6.966219,47924.0,1.106560,5.056397
4,facebook / cpc,Melihat Kantor Pelayanan Grab Driver Center,41155.0,11.630219,44072.0,43.809145,180.944233
...,...,...,...,...,...,...,...
151,facebook / cpc,"Edan, Ponsel Ini Punya Baterai 13.000 mAh",198.0,76.923077,215.0,1.837607,101.685393
152,facebook / cpc,Gempa Keras di Jakarta Terasa Lebih dari 1 Menit,180.0,20.000000,215.0,2.388889,233.640000
153,facebook / cpc,Jennifer Dunn dan Deretan Mobil Mewahnya,180.0,100.000000,215.0,23.888889,97.836735
154,facebook / cpc,Kasus Salah Baca Xiaomi Terulang Kembali,180.0,83.333333,215.0,1.990741,178.449438


## Check Duplicates and Unique Value

In [19]:
df['sourceMedium'].unique()
df[df.duplicated(subset=['pageTitle'])]

,sourceMedium,pageTitle,users,bounceRate,pageviews,pageviewsPerSession,avgTimeOnPage
103,google / cpc,Gempa Keras Guncang Jakarta,206.0,27.777778,323.0,1.993827,85.038627
114,google / cpc,"Kabar Duka, Sys NS Meninggal Dunia",81.0,42.857143,296.0,4.698413,10.051793
133,google / cpc,Andi Narogong Ungkap Peran Novanto cs di Proye...,117.0,0.000000,251.0,0.000000,84.765957
134,google / cpc,"Gempa 6,4 SR Guncang Jakarta",189.0,100.000000,251.0,27.888889,143.200000
142,google / cpc,"Messi Berjaket Go-Jek, Isyana Kini Jadi 'Korba...",9.0,0.000000,233.0,0.000000,24.607143


In [23]:
df[df['pageTitle'] == 'Gempa Keras Guncang Jakarta']
## duplicate cause 2 sources

df[df['pageTitle'] == 'Kabar Duka, Sys NS Meninggal Dunia']
df[df['pageTitle'] == 'Gempa 6,4 SR Guncang Jakarta']


,sourceMedium,pageTitle,users,bounceRate,pageviews,pageviewsPerSession,avgTimeOnPage
14,facebook / cpc,"Gempa 6,4 SR Guncang Jakarta",3717.0,50.0,4220.0,78.148148,256.062098
134,google / cpc,"Gempa 6,4 SR Guncang Jakarta",189.0,100.0,251.0,27.888889,143.200000


## Check Nulls

In [24]:
df[df.isnull().any(axis=1)]

,sourceMedium,pageTitle,users,bounceRate,pageviews,pageviewsPerSession,avgTimeOnPage
155,facebook / cpc,Netizen Ramai Teriakkan,NaN,NaN,NaN,NaN,NaN


In [27]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 155 entries, 0 to 154
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   sourceMedium         155 non-null    object 
 1   pageTitle            155 non-null    object 
 2   users                155 non-null    float64
 3   bounceRate           155 non-null    float64
 4   pageviews            155 non-null    float64
 5   pageviewsPerSession  155 non-null    float64
 6   avgTimeOnPage        155 non-null    float64
dtypes: float64(5), object(2)
memory usage: 9.7+ KB
